In [ ]:
import geopandas
import pathlib
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
slope_file = pathlib.Path(r"C:\Users\pearsonra\Documents\data\river_bathemetry\rec_domains\rec2_4\rec2_wshed_slopesV4.shp")
slope_data = geopandas.read_file(slope_file)

## Data sources

* Building polygons - all sources - https://data.linz.govt.nz/layer/101292-nz-building-outlines-all-sources/
* Populations - 2013 - https://datafinder.stats.govt.nz/layer/8437-population-by-meshblock-2013-census/
* Building points - not many included - https://data.linz.govt.nz/layer/50245-nz-building-points-topo-150k/

In [ ]:
slope_data